In [3]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from django.apps import apps as django_apps
from edc_appointment.models import Appointment as CaregiverAppointment
from flourish_caregiver.helper_classes.onschedule_helper import OnScheduleHelper
from edc_visit_schedule.site_visit_schedules import site_visit_schedules

In [5]:
subject_identifier = 'B142-040990786-1'
child_subject_identifier = 'B142-040990786-1-60'
child_count = 2

ssh = SubjectScheduleHistory.objects.filter(subject_identifier=subject_identifier)
for sh in ssh:
    onschedule_model_cls = django_apps.get_model(sh.onschedule_model)
    schedule_name = sh.schedule_name

    try:
        onschedule_obj = onschedule_model_cls.objects.get(
            subject_identifier=subject_identifier,
            child_subject_identifier=child_subject_identifier,
            schedule_name=schedule_name)
    except onschedule_model_cls.DoesNotExist:
        print(subject_identifier, 'No onschedule object', schedule_name)
    else:
        cohort_sch = CohortSchedules.objects.get(schedule_name=schedule_name)
        new_cohort_sch = CohortSchedules.objects.get(
            schedule_type=cohort_sch.schedule_type,
            cohort_name=cohort_sch.cohort_name,
            child_count=child_count)

        if cohort_sch.child_count != child_count:
            # Put on schedule for new schedule
            helper_cls = OnScheduleHelper(
                subject_identifier=subject_identifier,
                onschedule_datetime=onschedule_obj.onschedule_datetime,
                cohort=cohort_sch.cohort_name)

            visit_schedule, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
                onschedule_model=new_cohort_sch.onschedule_model, name=new_cohort_sch.schedule_name)
            new_onsch_cls = django_apps.get_model(new_cohort_sch.onschedule_model)
            try:
                new_onsch_cls.objects.get(
                    subject_identifier=subject_identifier, schedule_name=schedule.name)
            except new_onsch_cls.DoesNotExist:
                helper_cls.add_on_schedule(
                    schedule=schedule,
                    subject_identifier=subject_identifier,
                    instance=onschedule_obj,
                    schedule_name=new_cohort_sch.schedule_name,
                    base_appt_datetime=onschedule_obj.onschedule_datetime,
                    child_subject_identifier=child_subject_identifier,
                    onschedule_model_cls=new_onsch_cls)
                new_appts = CaregiverAppointment.objects.filter(
                    subject_identifier=subject_identifier, schedule_name=new_cohort_sch.schedule_name)
                new_appts.delete()

                appts = CaregiverAppointment.objects.filter(
                    subject_identifier=subject_identifier, schedule_name=schedule_name)
                for appt in appts:
                    visit = MaternalVisit.objects.get(appointment=appt)
                    appt.visit_schedule_name = visit_schedule.name
                    appt.schedule_name = schedule.name
                    appt.save_base(raw=True)
                    visit.visit_schedule_name = visit_schedule.name
                    visit.schedule_name = schedule.name
                    visit.save_base(raw=True)

            onschedule_obj.delete()
            sh.delete()

B142-040990786-1 No onschedule object b_sec1_schedule1
B142-040990786-1 No onschedule object b_sec_quart1_schedule1
